In [57]:
# To mount the google drive to google colab:

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [58]:
import pandas as pd

# read the dataset:

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv', error_bad_lines=False, engine='python')

<ipython-input-58-0f539d1e80e4>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv', error_bad_lines=False, engine='python')
Skipping line 13535: field larger than field limit (131072)
Skipping line 19783: field larger than field limit (131072)
Skipping line 13541: Expected 5 fields in line 13541, saw 6
Skipping line 19782: Expected 5 fields in line 19782, saw 7
Skipping line 19783: Expected 5 fields in line 19783, saw 10
Skipping line 19785: Expected 5 fields in line 19785, saw 14
Skipping line 19787: Expected 5 fields in line 19787, saw 8
Skipping line 19789: Expected 5 fields in line 19789, saw 7
Skipping line 19790: Expected 5 fields in line 19790, saw 15
Skipping line 19791: Expected 5 fields in line 19791, saw 9
Skipping line 19792: Expected 5 fields in line 19792, saw 9
Skipping line 19793: Expected 5 fields in line 19793

In [59]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [60]:
# check for the null values:

df.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [61]:
# drop all the NaN values:
df = df.dropna()

In [62]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [63]:
# checking the unique labels:

df["label"].unique()

array(['1', '0', ' как люди воспринимают своё положение.',
       ' чтобы это была дорога с двусторонним движением.'], dtype=object)

In [64]:
df["label"].value_counts()


0                                                   10361
1                                                    7922
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: label, dtype: int64

In [65]:
# drop all the rows which doesnt not contain 0 or 1:
df = df[df["label"].isin(['0','1'])]

In [66]:
df["label"].value_counts()


0    10361
1     7922
Name: label, dtype: int64

In [85]:
df.shape

(18283, 5)

In [86]:
# Get the independent features:

X = df.drop("label", axis=1)

In [87]:
# Get the dependent features:
y = df["label"]

In [88]:
print("Checking the shape of dataframe", X.shape)

Checking the shape of dataframe (18283, 4)


In [89]:
print("Checking the shape of y ", y.shape)

Checking the shape of y  (18283,)


In [71]:
pip install tensorflow==2.8.2

ERROR: Operation cancelled by user


In [72]:
import tensorflow as tf

tf.__version__

'2.8.2'

In [73]:
# import the tensorflow libraries:

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [90]:
# Define the vocab size:

voc_size = 5000

In [91]:
# One hot representation:

messages = X.copy()

messages["title"][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [92]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [93]:
# reset the index:
messages.reset_index(inplace=True)

In [94]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18278,20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18279,20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18280,20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18281,20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [95]:
import nltk
import re
from nltk.corpus import stopwords


nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [96]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18278,20817,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18279,20818,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18280,20819,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18281,20820,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [97]:
# data preprocessing:

from nltk.stem.porter import PorterStemmer # for the stemming purpose

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages["title"][i])
  review = review.lower()
  # split the sentences based on the space:
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
  review = ' '.join(review)
  corpus.append(review)




In [98]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
# checking an example of corpus:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [99]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [100]:
onehot_repr[1]

[4484, 2169, 4198, 3493, 4174, 3504, 2653]

In [101]:
# Embedding Representation:

sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding="post", maxlen = sent_length)
print(embedded_docs)

[[2623 1185 4602 ...    0    0    0]
 [4484 2169 4198 ...    0    0    0]
 [2225 2777 1191 ...    0    0    0]
 ...
 [1969 1439 3769 ...    0    0    0]
 [ 218 4984 1822 ...    0    0    0]
 [1008  123   54 ...    0    0    0]]


In [25]:
embedded_docs[1]

array([4484, 2169, 4198, 3493, 4174, 3504, 2653,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [26]:
embedded_docs[0]

array([2623, 1185, 4602, 1234,  814, 2834, 1238, 1214, 2567, 3446,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [102]:
# Creating a model:

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
len((embedded_docs)), y.shape

(18283, (18283,))

In [112]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)



In [113]:
X_final.shape, y_final.shape

((18283, 20), (18283,))

In [111]:
y.unique()

y = y.astype(int)

In [115]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final,test_size=0.33, random_state=42)

In [117]:
y_test

array([0, 1, 1, ..., 1, 1, 1])

In [118]:
# Model Training:

model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 66ms/step - loss: 0.3109 - accuracy: 0.8475 - val_loss: 0.1992 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 8s 42ms/step - loss: 0.1385 - accuracy: 0.9447 - val_loss: 0.1967 - val_accuracy: 0.9223
Epoch 3/10
192/192 [==============================] - 12s 63ms/step - loss: 0.0961 - accuracy: 0.9649 - val_loss: 0.2213 - val_accuracy: 0.9201
Epoch 4/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0737 - accuracy: 0.9738 - val_loss: 0.2761 - val_accuracy: 0.9143
Epoch 5/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0539 - accuracy: 0.9821 - val_loss: 0.2898 - val_accuracy: 0.9103
Epoch 6/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0398 - accuracy: 0.9870 - val_loss: 0.3489 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0309 - accuracy: 0.9902 - val_loss: 0.3593 - val_accuracy: 0

In [119]:
# Adding a  Dropout:

from tensorflow.keras.layers import Dropout
# creating model:

embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [120]:
# Performance metrics and accuracy:

y_pred = model.predict(X_test)

In [121]:
y_pred = np.where(y_pred > 0.6, 1, 0) ### AUC ROC Curve

In [122]:
from sklearn.metrics import confusion_matrix

In [123]:
confusion_matrix(y_test, y_pred)

array([[3415,    0],
       [2619,    0]])

In [124]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.565959562479284

In [125]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3415
           1       0.00      0.00      0.00      2619

    accuracy                           0.57      6034
   macro avg       0.28      0.50      0.36      6034
weighted avg       0.32      0.57      0.41      6034



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
